In [48]:
import pandas as pd
#libraries to connect to API
import json
import requests
pd.options.display.max_rows = 100

In [49]:
#function to load API Key locally stored
def get_keys(path):
    with open(path) as f:
        return json.load(f)

## US CENSUS BUREAU API

I used the US Census Bureau API to extract several variables for my study. 

API Key request: https://api.census.gov/data/key_signup.html

I extracted the data from the **American Community Survey 5 Year Data (2009-2018)**. This survey covers topics about social, economic, demographic and housing characteristics of the U.S. population. There are over 20,000 variables in this dataset, but not many of the variables are available down to the county-level. 

**ACS 5-Year:** https://www.census.gov/data/developers/data-sets/acs-5year.html


From the Technical Documentation, I used the Table List to find table IDs and the Table Shells to get the Unique ID to the exact variables I wanted to include in the study. https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html?#  -AND-  https://api.census.gov/data/2018/acs/acs5/variables.html

I added an "E" at the end of each variable ID to retrieve the "estimate" value of the variable. Also, I divided my API requests in groups < 50 variables since the US Census API has that restriction (you will get an error message if you include more than 50 variables).


In [50]:
#import API Key from local directory
#API keys are sensitive and should not be hard coded to your notebook
keys = get_keys("/Users/franc/.secret/USCensus.json")
api_key = keys['api_key'] 

In [51]:
variable_list1 = (
                    'B01002_002E,B01002_003E,B08301_002E,B08301_010E,B08301_016E,'
                    'B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E,B08303_001E,B08303_002E,'
                    'B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,'
                    'B08303_010E,B08303_011E,B08303_012E,B08303_013E,B09002_001E,B11001_001E,B11001_003E,'
                    'B11001_005E,B11001_006E,B11001_008E,B11001_009E,B11017_002E,B12001_002E,B12001_003E,B12001_004E,'
                    'B12001_009E,B12001_010E,B12001_011E,B12001_012E,B12001_013E,B12001_018E,B12001_019E'
                )

variable_list2 = (
                    'B12007_001E,B12007_002E,B12504_002E,'
                    'B12504_006E,B13016_002E,B15002_003E,B15002_011E,'
                    'B15002_015E,B15002_016E,B15002_018E,B15002_020E,B15002_028E,B15002_032E,B15002_033E,'
                    'B15002_035E,B17001_001E,B17001_031E,B17001_002E,B19013_001E,B19058_001E,B19058_002E,'
                    'B19083_001E,B19301_001E,B23020_002E,B23020_003E,B23025_001E,B23025_004E,B25001_001E,B25003_002E,'
                    'B25003_003E,B25018_001E,B25058_001E,B25105_001E,B28010_007E,B28011_008E'
                ) 

In [52]:
url = 'https://api.census.gov/data/2018/acs/acs1.json?get=NAME,'+ variable_list1 +'&for=county:*'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),}
response = requests.get(url, headers=headers).json() #convert data to a json format
ACS_data1 = pd.DataFrame(response)

url = 'https://api.census.gov/data/2018/acs/acs1.json?get=NAME,'+ variable_list2 +'&for=county:*'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),}
response = requests.get(url, headers=headers).json() #convert data to a json format
ACS_data2 = pd.DataFrame(response)

##DATA/TABLE FORMATTING##
ACS_data1.columns = ACS_data1.iloc[0] #take column headers from row0
ACS_data2.columns = ACS_data2.iloc[0] #take column headers from row0
ACS_data1.drop(ACS_data1.index[0], inplace=True) #drop row 0
ACS_data2.drop(ACS_data2.index[0], inplace=True) #drop row 0
ACS_alldata = ACS_data1.merge(ACS_data2, how='left', on=['NAME','state','county']) #Merge data into a single table
ACS_alldata.drop(labels=['state', 'county'], inplace=True, axis=1) #drop unnecessary columns

countyname = ACS_alldata['NAME'].str.split(', ', expand=True) #Split name column by comma
ACS_alldata['county'] = countyname[0]
ACS_alldata['state_name'] = countyname[1]
ACS_alldata.loc[ACS_alldata['county'].str.contains('county|County|Parish|parish'),'county'] = ACS_alldata['county'].str[:-7] #remove the word "county" from county column

ACS_alldata = ACS_alldata.assign(newcounty=ACS_alldata['county'].str.replace('.', '')) #remove '.' from county names if any
ACS_alldata.drop(labels=['county'], inplace=True, axis=1)
ACS_alldata.rename(columns={'newcounty': "county"}, inplace=True)

rowstodrop = ACS_alldata.loc[ACS_alldata['state_name'].isin(['Puerto Rico', 'Alaska', 'Hawaii'])] #exclude from the study
ACS_alldata = ACS_alldata.drop(rowstodrop.index) #exclude from the study

ACS_alldata.loc[ACS_alldata['county'] == "Doña Ana",'county'] = 'DONA ANA' #for consistency across sources
ACS_alldata['county'] = ACS_alldata['county'].apply(lambda x: x.upper()) #make all county names upper case for consistency

When you download data from the US Census Bureau API, the columns come with the variable ID rather than with a column name/description. I prepared a csv file with Variable ID and Column Name to assign an "appropriate" column name to each variable. 

In [53]:
ACS_headers = pd.read_csv("DATA/ACSNew_headers.csv") #file with new column names
ACS_headers.set_index('Old_header', inplace=True)
header_dict = ACS_headers.to_dict('dict') #create a dict with new column names
ACS_alldata.rename(columns=header_dict['New_header'], inplace=True)

In [54]:
ACS_alldata.isna().sum() #Check the number of NaNs in each column

0
NAME                                                                                    0
median age male                                                                         0
median age female                                                                       0
means of transportation to work: car, truck, or van                                   451
means of transportation to work: public transportation (excluding taxicab)            451
means of transportation to work: taxicab                                              451
means of transportation to work: motorcycle                                           451
means of transportation to work: bicycle                                              451
means of transportation to work: walked                                               451
means of transportation to work: other means                                          451
means of transportation to work: worked at home                                       451
total tr

In [55]:
#drop columns with over 50% missing data
NaNColumns = ['means of transportation to work: car, truck, or van', 'means of transportation to work: public transportation (excluding taxicab)', 
              'means of transportation to work: taxicab', 'means of transportation to work: motorcycle', 'means of transportation to work: bicycle',
              'means of transportation to work: walked', 'means of transportation to work: other means', 'means of transportation to work: worked at home',
              'multigenerational households', 'median age at first marriage male', 'median age at first marriage female', 'median duration of current marriage in years male',
              'median duration of current marriage in years female', ]
ACS_alldata.drop(labels=NaNColumns, inplace=True, axis=1) #drop columns with over 50% missing data

#Assign median state value to missing fields 
null_indexes = ACS_alldata.loc[ACS_alldata['own children under 18 years by family type and age'].isnull()].index
for i, indx in enumerate(null_indexes):
    children = ACS_alldata[ACS_alldata['state_name'] == ACS_alldata.loc[indx]['state_name']]['own children under 18 years by family type and age'].median()
    ACS_alldata.at[indx, 'own children under 18 years by family type and age'] = children

null_indexes = ACS_alldata.loc[ACS_alldata['mean usual hours worked male'].isnull()].index
for i, indx in enumerate(null_indexes):
    hours = ACS_alldata[ACS_alldata['state_name'] == ACS_alldata.loc[indx]['state_name']]['mean usual hours worked male'].median()
    ACS_alldata.at[indx, 'mean usual hours worked male'] = hours   

null_indexes = ACS_alldata.loc[ACS_alldata['mean usual hours worked female'].isnull()].index
for i, indx in enumerate(null_indexes):
    hours = ACS_alldata[ACS_alldata['state_name'] == ACS_alldata.loc[indx]['state_name']]['mean usual hours worked female'].median()
    ACS_alldata.at[indx, 'mean usual hours worked female'] = hours 
    

In [56]:
ACS_alldata.isna().sum() #Check the number of NaNs in each column

0
NAME                                                                                  0
median age male                                                                       0
median age female                                                                     0
total travel time to work                                                             0
travel time to work: less than 5 minutes                                              0
travel time to work: 5 to 9 minutes                                                   0
travel time to work: 10 to 14 minutes                                                 0
travel time to work: 15 to 19 minutes                                                 0
travel time to work: 20 to 24 minutes                                                 0
travel time to work: 25 to 29 minutes                                                 0
travel time to work: 30 to 34 minutes                                                 0
travel time to work: 35 to 39 

In [58]:
#Export resulting table to csv
ACS_alldata.to_csv('DATA/ACS_alldata.csv')

## WEATHER DATA

### Web Scraping 

In [ ]:
#Import reference data to the notebook
#This is the list of counties for the study
county_selection = pd.read_csv("MENUISM3/county_selection.csv")

#create lists for loop
county_name = list(county_selection['county'])
state_name = list(county_selection['state_name'])
len(state_name)

webscr_county = county_name[787:823]
webscr_state = state_name[787:823]

In [ ]:
#     user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10"
#     user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'
#     user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'

In [ ]:
randomtimes = [0.5, 0.75] #these are seconds for the sleep method
df = pd.DataFrame(columns=['county', 'state_name', 'month', 'high/low', 'rain']) #initialize dataframe
for j,zipi in enumerate(webscr_county):
    county = webscr_county[j].replace(' ', '%20')
    state = webscr_state[j] .replace(' ', '%20')
    url = "https://www.google.com/search?q=Average+monthly+temperature+" + county + "+county+" + state
    user_agent =  'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    month = soup.find_all("span", {"role":"gridcell"})
    print(j)

    i = 0
    while i < len(month):
        txtmonth = month[i].text
        txttemp = month[i+1].text
        txtrain = month[i+2].text
        dctionary = {'county': [webscr_county[j]],'state_name': [webscr_state[j]],'month':[txtmonth],'high/low':[txttemp], 'rain':[txtrain]}
        df = pd.concat([df,pd.DataFrame(dctionary)], ignore_index=True, sort=False)
        i = i + 3

In [ ]:
df.to_csv('DATA/temperaturedata.csv')

Data Cleanning

In [ ]:
#import web scrapped data
temperature_data = pd.read_csv("DATA/temperaturedata.csv")
temperature_data.drop(labels=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1) #drop unnecessary columns
temperature_data.drop_duplicates(keep='first', inplace=True) #drop duplicate rows

In [ ]:
#check if any county does not have 12 months
month_check = temperature_data.groupby(['county', 'state_name'], as_index=False)['month'].count()
month_check.rename(columns={'month':'month_count'}, inplace=True)

month_check.loc[month_check['month_count'] != 12]

In [ ]:
#inspect the counties with differences 
temperature_data.loc[temperature_data['county'] == 'Chelan']
temperature_data.loc[temperature_data['county'] == 'Terrebonne']

In [ ]:
#drop data after inspection
drop_rows = temperature_data.loc[temperature_data['county'].isin(['Chelan', 'Terrebonne'])]
temperature_data = temperature_data.drop(drop_rows.index)

In [ ]:
#Check if I have all the info for the seleted counties
keys = ['county', 'state_name']
i1 = month_check.set_index(keys).index
i2 = county_selection.set_index(keys).index
missing = county_selection[~i2.isin(i1)]

missing

In [ ]:
#split temperature in high/low
temp = temperature_data['high/low'].str.split('/', expand=True)
temperature_data['highCelsius'] = temp[0] 
temperature_data['lowCelsius'] = temp[1]

#remove the degree sign from temperature
temperature_data['highCelsius'] = temperature_data.apply(lambda x: x['highCelsius'].strip(), axis=1)
temperature_data['lowCelsius'] = temperature_data.apply(lambda x: x['lowCelsius'].strip(), axis=1)

temperature_data['highCelsius'] = temperature_data.apply(lambda x: x['highCelsius'][:-1], axis=1)
temperature_data['lowCelsius'] = temperature_data.apply(lambda x: x['lowCelsius'][:-1], axis=1)

#remove 'dias' word from rain days
temperature_data['rain'] = temperature_data.apply(lambda x: x['rain'].strip(), axis=1)
temperature_data['rain'] = temperature_data.apply(lambda x: x['rain'][:-5], axis=1)
temperature_data.rename(columns={'rain': "days_of_rain"}, inplace=True)

#remove columns
temperature_data.drop(labels=['high/low'], inplace=True, axis=1)

In [ ]:
temperature_data['month'].replace({"enero": "January", "febrero": "February", "marzo": "March", "abril": "April", "mayo": "May", "junio": "June", "julio": "July", "agosto": "August", "setiembre": "September", "octubre": "October", "noviembre": "November", "diciembre": "December"}, inplace=True)

In [ ]:
#export data to folder
temperature_data.to_csv("DATA\Rain&Temperaturedata.csv")